In [ ]:
!pip install transformers
!pip install sentencepiece

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.5 MB 9.2 MB/s 
     |████████████████████████████████| 7.6 MB 33.7 MB/s 
     |████████████████████████████████| 163 kB 58.1 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.3 MB 14.7 MB/s 


In [ ]:
import nltk
import re
import argparse
import pandas as pd
from transformers import MarianMTModel, MarianTokenizer

In [ ]:
import transformers, torch
import pandas as pd
from tqdm import tqdm
from transformers import MarianMTModel, MarianTokenizer


def main():
  def translate(texts, model, tokenizer, language="es"):
      template = lambda text: f"{text}"
      src_texts = [template(text) for text in texts]
      encoded = tokenizer.prepare_seq2seq_batch(src_texts, return_tensors='pt').to(device)
      translated = model.generate(**encoded, do_sample=True, max_length=256)
      translated_texts = tokenizer.batch_decode(translated, skip_special_tokens=True)
      return translated_texts

  def back_translate(texts, source_lang="es", target_lang="en"):
      fr_texts = translate(texts, target_model, target_tokenizer, language=target_lang)
      back_translated_texts = translate(fr_texts, en_model, en_tokenizer, language=source_lang)
      return back_translated_texts

  def generate_backtranslate_df_csv(input_dataframe, filename):
    backtranslated_texts = []

    for i in tqdm(range(0, len(input_dataframe), batch_size)):
        comments = input_dataframe[i:i+batch_size]
        aug_text = back_translate(comments['text'].tolist(), source_lang="en", target_lang="es")
        #print("\n")
        #print(aug_text)
        backtranslated_texts.extend(aug_text)
        torch.cuda.empty_cache()

    output_dataframe = pd.DataFrame(columns=['text', 'label'])
    output_dataframe['label'] = input_dataframe['label']
    output_dataframe['text'] = backtranslated_texts


    output_dataframe.to_csv(filename + ".csv", encoding="utf-8", index=False, header=False)
    print("Generated backtranslated "+ filename + ".csv successfully")

  batch_size = 128
  device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

  target_model_name = 'Helsinki-NLP/opus-mt-en-es'
  target_tokenizer = MarianTokenizer.from_pretrained(target_model_name)
  target_model = MarianMTModel.from_pretrained(target_model_name).cuda()

  en_model_name = 'Helsinki-NLP/opus-mt-es-en'
  en_tokenizer = MarianTokenizer.from_pretrained(en_model_name)
  en_model = MarianMTModel.from_pretrained(en_model_name).cuda()

  df_train = pd.read_csv("train.csv", sep=',', names=['text', 'label'])
  df_dev = pd.read_csv("dev.csv", sep=',', names=['text', 'label'])
  df_test = pd.read_csv("test.csv", sep=',', names=['text', 'label'])

  #generate_backtranslate_df_csv(df_dev, "dev_backtranslated_es")
  generate_backtranslate_df_csv(df_train, "train_backtranslated_es")
  #generate_backtranslate_df_csv(df_test, "test_backtranslated_es")


main()

Downloading:   0%|          | 0.00/802k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/826k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.59M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/44.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.47k [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/44.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.44k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/312M [00:00<?, ?B/s]

  0%|          | 0/96 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:3672: FutureWarning: 
`prepare_seq2seq_batch` is deprecated and will be removed in version 5 of HuggingFace Transformers. Use the regular
`__call__` method to prepare your inputs and targets.

Here is a short example:

model_inputs = tokenizer(src_texts, text_target=tgt_texts, ...)

If you either need to use different keyword arguments for the source and target texts, you should do two calls like
this:

model_inputs = tokenizer(src_texts, ...)
labels = tokenizer(text_target=tgt_texts, ...)
model_inputs["labels"] = labels["input_ids"]

See the documentation of your specific tokenizer for more details on the specific arguments to the tokenizer of choice.
For a more complete example, see the implementation of `prepare_seq2seq_batch`.

  warnings.warn(formatted_warning, FutureWarning)
100%|██████████| 96/96 [24:41<00:00, 15.44s/it]

Generated backtranslated train_backtranslated_es.csv successfully


In [ ]:
def clean(text, remove_stopwords, backtranslate_preprocess):
    text = re.sub(r'URL', "", text)  # Remove URL's
    text = re.sub(r'@USER', "", text)  # Remove @users

    text = text.lower()  # Lowercase all text
    text = re.sub(r"what's", "what is", text)
    text = re.sub(r"i'm", "i am", text)
    text = re.sub(r"'re", " are", text)
    text = re.sub(r"\'d", " would", text)
    text = re.sub(r"i'm", "i am", text)
    text = re.sub(r"n't", " not", text)
    text = re.sub(r"\'scuse", "excuse", text)
    text = re.sub(r"\'ll", " will", text)
    text = re.sub(r"\'ve", " have", text)
    text = re.sub(r"can't", "cannot", text)
    text = re.sub(r"doesn't", "does not", text)
    if backtranslate_preprocess:
        text = re.sub(r"[^a-zA-Z]", " ", text)  # Removes special chars other than(?!.,')
        text = re.sub(r"\s{2,}", " ", text)  # Remove tabs double spaces newlines etc
    else:
        text = re.sub(r"[.,?!']{2,}", "", text)  # Only leave .?!' with only 1 char
        text = re.sub(r'\d+', "", text)  # Remove numbers
        text = re.sub(r'/#\w+\s*/', "", text)  # Remove hashtags
        text = re.sub(r"[^a-z.?,!'A-Z]", " ", text)  # Removes special chars other than(?!.,')
        text = re.sub(r"\s{2,}", " ", text)  # Remove tabs double spaces newlines etc
        if remove_stopwords:
            text = text.split()
            text = " ".join([word for word in text if not word in stop_word])  # Remove stop words
    return text


def clean_dataframe(df, remove_stopwords, backtranslate_preprocess):
    df = df.apply(lambda x: clean(x, remove_stopwords, backtranslate_preprocess))
    return df

In [ ]:
df_train = pd.read_csv('preprocessed_data/train.csv', sep=',', names=['text', 'label'])
df_dev = pd.read_csv('preprocessed_data/dev.csv', sep=',', names=['text', 'label'])
df_test = pd.read_csv('preprocessed_data/test.csv', sep=',', names=['text', 'label'])

df_train_backtranslated = pd.read_csv('preprocessed_data_backtranslated/train_backtranslated_es.csv', sep=',',names=['text', 'label'])
df_dev_backtranslated = pd.read_csv('preprocessed_data_backtranslated/dev_backtranslated_es.csv', sep=',',names=['text', 'label'])
df_test_backtranslated = pd.read_csv('preprocessed_data_backtranslated/test_backtranslated_es.csv', sep=',', names=['text', 'label'])

# df_train_backtranslated = df_train_backtranslated['text'].str.lower()
# df_dev_backtranslated = df_dev_backtranslated['text'].str.lower()
# df_test_backtranslated = df_test_backtranslated['text'].str.lower()

df_train_backtranslated['text'] = clean_dataframe(df_train_backtranslated['text'], False, True)
df_dev_backtranslated['text'] = clean_dataframe(df_dev_backtranslated['text'], False, True)
df_test_backtranslated['text'] = clean_dataframe(df_test_backtranslated['text'], False, True)

# df_train['text'] = clean_dataframe(df_train['text'], False, True)
# df_dev['text'] = clean_dataframe(df_dev['text'], False, True)
# df_test['text'] = clean_dataframe(df_test['text'], False, True)


merged_train = pd.concat([df_train, df_train_backtranslated]).drop_duplicates().reset_index(drop=True)
merged_dev = pd.concat([df_dev, df_dev_backtranslated]).drop_duplicates().reset_index(drop=True)
merged_test = pd.concat([df_test, df_test_backtranslated]).drop_duplicates().reset_index(drop=True)
